![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/31.1.Porting_QA_Models_From_Text_Generator_Backbone.ipynb)


# **Porting Question Answering Models From Text Generator Backbone**


## Colab Setup

📌To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs==5.1.0

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

In [ ]:
spark


The `loadMedicalTextGenerator` function allows for the conversion of a model from `MedicalTextGenerator` to `MedicalQuestionAnswering`. By utilizing this function, you can seamlessly repurpose the existing model to generate answers for medical-related questions.

```
med_text_gen = sparknlp_jsl.annotators.MedicalTextGenerator.pretrained()
med_text_gen.write().overwrite().save("/tmp/med_text")
med_qa = sparknlp_jsl.annotators.MedicalQuestionAnswering.loadMedicalTextGenerator("/tmp/med_text", spark)
```

## Download TextGenerator

In [ ]:
med_text_generator  = medical.TextGenerator.pretrained("text_generator_biomedical_biogpt_base", "en", "clinical/models")\
    .setInputCols("document_prompt")\
    .setOutputCol("answer")


text_generator_biomedical_biogpt_base download started this may take some time.
[OK!]


In [ ]:
med_text_generator.write().overwrite().save("./med_text")

## Load Model

In [ ]:
document_assembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = medical.QuestionAnswering.loadMedicalTextGenerator("./med_text", spark)\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Question: {QUESTION}. Context: {DOCUMENT} ")\
    .setMaxNewTokens(70)\
    .setTopK(1)\
    .setOutputCol("answer")\
    .setQuestionType("short")

pipeline = nlp.Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([["",""]]).toDF("question", "context")

model = pipeline.fit(empty_data)

**LightPipelines**

In [ ]:
med_qa.setCustomPrompt("Question: {QUESTION}. Context: {DOCUMENT} ")


pipeline = nlp.Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([["",""]]).toDF("question", "context")

model = pipeline.fit(empty_data)

In [ ]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "How is transgastric endoscopic performed?"

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['How is transgastric endoscopic performed?'],
  'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insuffl

In [ ]:
light_result[0]["answer"]

['question: How is transgastric endoscopic performed?. context: { document }. Introduction: The aim of this study was to evaluate the technical feasibility and safety of transgastric endoscopic surgery ( TES ) in the treatment of gastric lesions. Methods: A retrospective analysis of a prospectively collected database was performed. All patients who underwent TES for gastric lesions between January 2010 and December 2015 were included. Results: A total of']